In [18]:
pwd
#!cd ../src

NameError: name 'pwd' is not defined

In [20]:
cd ../src/

/tf/capstone2/data/app/breast-cancer-cnn/src


In [4]:
#!pip list | grep tensorflow
#!pip install --upgrade pip
#!pip install pillow


In [21]:
#!pip install --upgrade tensorflow

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential, save_model, load_model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras import layers, preprocessing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, \
    Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
#from tensorflow.keras.callbacks import TensorBoard
import numpy as np

In [21]:
from image_pipeline import *
from image_convolv import *
from bc_plotting import *
from bc import *

In [22]:
# needs a version higher than tf 2.0, perhaps high as 2.3 to use 'SavedModel' format
# then refer to the entire folder
# model_path='../models/best_model.hdf5'
basic_model=tf.keras.models.load_model('../models/saved_model.h5')

In [23]:
# i can do lots of fun things w my reconstructed model
basic_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 153, 234, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 149, 230, 64)      51264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 76, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 72, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 24, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 46080)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2

In [12]:
len(basic_model.weights) #12
model.basic_model[:1]  # shape=(5, 5, 3, 32)
arr = np.array(basic_model.weights)
arr.shape  # (12,)

NameError: name 'model' is not defined

In [18]:
basic_model.layers

In [19]:
def print_model_properties(model, indices = 0):
     for i, layer in enumerate(model.layers[indices:]):
        print("Layer {} | Name: {} | Trainable: {}".format(i+indices, layer.name, layer.trainable))
        
print_model_properties(basic_model)

Layer 0 | Name: conv2d | Trainable: True
Layer 1 | Name: conv2d_1 | Trainable: True
Layer 2 | Name: max_pooling2d | Trainable: True
Layer 3 | Name: conv2d_2 | Trainable: True
Layer 4 | Name: max_pooling2d_1 | Trainable: True
Layer 5 | Name: flatten | Trainable: True
Layer 6 | Name: dense | Trainable: True
Layer 7 | Name: dense_1 | Trainable: True
Layer 8 | Name: dense_2 | Trainable: True


In [20]:
from tensorflow.keras.applications import Xception

In [21]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model

def create_transfer_model(input_size, n_categories, weights = 'imagenet'):
        # note that the "top" is not included in the weights below
        base_model = Xception(weights=weights,
                          include_top=False,
                          input_shape=input_size)
        
        model = base_model.output
        model = GlobalAveragePooling2D()(model)
        predictions = Dense(n_categories, activation='softmax')(model)
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model
    
transfer_model = create_transfer_model(input_size=(100,100,3), n_categories=5)

83689472/83683744 [==============================] - 3s 0us/step


In [24]:
def print_model_properties(model, indices = 0):
     for i, layer in enumerate(model.layers[indices:]):
        print("Layer {} | Name: {} | Trainable: {}".format(i+indices, layer.name, layer.trainable))
        
print_model_properties(transfer_model)

Layer 0 | Name: input_1 | Trainable: False
Layer 1 | Name: block1_conv1 | Trainable: False
Layer 2 | Name: block1_conv1_bn | Trainable: False
Layer 3 | Name: block1_conv1_act | Trainable: False
Layer 4 | Name: block1_conv2 | Trainable: False
Layer 5 | Name: block1_conv2_bn | Trainable: False
Layer 6 | Name: block1_conv2_act | Trainable: False
Layer 7 | Name: block2_sepconv1 | Trainable: False
Layer 8 | Name: block2_sepconv1_bn | Trainable: False
Layer 9 | Name: block2_sepconv2_act | Trainable: False
Layer 10 | Name: block2_sepconv2 | Trainable: False
Layer 11 | Name: block2_sepconv2_bn | Trainable: False
Layer 12 | Name: conv2d | Trainable: False
Layer 13 | Name: block2_pool | Trainable: False
Layer 14 | Name: batch_normalization | Trainable: False
Layer 15 | Name: add | Trainable: False
Layer 16 | Name: block3_sepconv1_act | Trainable: False
Layer 17 | Name: block3_sepconv1 | Trainable: False
Layer 18 | Name: block3_sepconv1_bn | Trainable: False
Layer 19 | Name: block3_sepconv2_act |

In [23]:
def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True
        
change_trainable_layers(transfer_model, 132)

In [29]:
# Here's where I need my data
import sys  
sys.path.insert(0, '../src')

from bc import *
from image_pipeline import *

In [30]:
ip = run_pipeline()
# reads images from root_dir where last subdir is 200X, returns Image_Pipeline obj

first image shape (460, 700, 3)
first names entry SOB_B_A-14-22549AB-200-001.png
images_list should have list of 2013 patients x images, filenames of shape 460 
first image shape (460, 700, 3)
images_filename_list should have list of 2013 patients * images
first names entry SOB_B_A-14-22549AB-200-001.png
{'slide_id': '22549AB', 'tumor_type': 'A', 'biopsy_procedure': 'SOB', 'tumor_class': 'B', 'seq': '001', 'year': '14', 'image_size': (460, 700, 3), 'mag': '200'}


In [ ]:
transfer_model.compile(optimizer='SGD', loss=['categorical_crossentropy'], metrics=['accuracy'])
mdl_check_trans = ModelCheckpoint(filepath='models/best_trans_model.hdf5',
                            save_best_only=True)
transfer_model.fit_generator(generator=train_small, 
                    validation_data=val_small,
                    epochs=2, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_steps=validation_steps,
                    callbacks=[mdl_check_trans, tensorboard])